Install qlatent is required

In [ ]:
# !pip install qlatent

In [2]:
import importlib

from qlatent.qmnli.qmnli import *
from qlatent.qmnli.qmnli import _QMNLI, QMNLI
# importlib.reload(_QMNLI)

import torch
device = 0 if torch.cuda.is_available() else -1
print(device)

0


In [3]:
# softmax_files = [False, True]
# softmax_files = [True]

def split_question(Q, index, scales, softmax, filters):
  result = []
  for s in scales:
    q = QCACHE(Q(index=index, scale=s))
    for sf in softmax:
      for f in filters:
        if sf:            
            qsf = QSOFTMAX(q,dim=[index[0], s])
            qsf_f = QFILTER(qsf,filters[f],filtername=f)
            print((index, s),sf,f)
            result.append(qsf_f)
            
#             qsf = QSOFTMAX(q,dim=s)
#             qsf_f = QFILTER(qsf,filters[f],filtername=f)
#             print(s,sf,f)
#             result.append(qsf_f)
            
#             qsf = QSOFTMAX(q,dim=index[0])
#             qsf_f = QFILTER(qsf,filters[f],filtername=f)
#             print(index[0],sf,f)
#             result.append(qsf_f)
        else:
            qsf = QPASS(q,descupdate={'softmax':''})
            qsf_f = QFILTER(qsf,filters[f],filtername=f)
            print(s,sf,f)
            result.append(qsf_f)
  return result

def dict_pos_neg(pos, neg, w):
  return dict(dict_same_weight(1.0*w/len(pos),pos), **dict_same_weight(-1.0*w/len(neg),neg))

def print_permutations(q):
#     for q in Q1s:
    W = q._pdf['W']
    print(q._descriptor)
    for i, (kmap, w) in enumerate(zip(q._keywords_map, W)):
        context = q._context_template.format_map(kmap)
        answer = q._answer_template.format_map(kmap)
#         sexisem_score = sexisem_classifier(context.strip('.') + ' ' +answer)
        print(f'{i}.',context ,'->', answer, w)
#     break


frequency_weights:SCALE = {
    'never':-4,
    'very rarely':-3,
    'seldom':-2,
    'rarely':-2,
    'frequently':2,
    'often':2,
    'very frequently':3,
    'always':4,    
}
    
    
binary_frequency_weights = {k: 1 if v > 0 else -1 for k, v in frequency_weights.items()}
rbinary_frequency_weights = {k: -1 if v > 0 else 1 for k, v in frequency_weights.items()}
frequency_pos = [k for k, v in frequency_weights.items() if v > 0]
frequency_neg = [k for k, v in frequency_weights.items() if v < 0]

In [4]:
# p = 'valhalla/distilbart-mnli-12-6'
p = "typeform/distilbert-base-uncased-mnli"
mnli = pipeline("zero-shot-classification",device=device, model=p)
mnli.model_identifier = p


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


# PHQ 2

In [5]:
from qpsychometric.mental_health.patient_health_questionnaire.phq2_qmnli import phq2_qmnli

In [11]:
for Q in phq2_qmnli:
    Qs = split_question(Q,
                          index=Q()._index,
                          scales=[Q()._scale],
                          softmax=[True],
                          filters={'unfiltered':{},
                                   "positiveonly":Q().get_filter_for_postive_keywords()
                                  },
                        )
    # softmax
    Qs[0].run(mnli).report()
#     # softmax + positive only
#     Qs[0].run(mnli).mean_score()

(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
Query time: 0.02854323387145996
Mean score unfiltered [-2.0..2.0]: -0.11203825548363966
Internal consistency (silhouette, correlation) for unfiltered: 0.9813957814760023
Internal consistency (Calinski&Harabasz)  for unfiltered: 120.51797373834366
Internal consistency (Davies&Bouldin) for unfiltered: 0.11341041635005845


index = ['emotion']
{'emotion', 'intensifier'} {'intensifier'} {'emotion'}
[]


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
pleasure,0.0000,0.0158,0.0058,0.0005,0.2565,0.2458,0.2460,0.2296
satisfaction,0.0000,0.0083,0.0028,0.0002,0.2394,0.2368,0.2488,0.2638
displeasure,0.3320,0.1983,0.2359,0.2303,0.0001,0.0031,0.0002,0.0000
dissatisfaction,0.2619,0.2247,0.2542,0.2552,0.0002,0.0035,0.0003,0.0000
little interest,0.1644,0.2901,0.2655,0.2733,0.0001,0.0063,0.0002,0.0001
no pleasure,0.2480,0.2485,0.2390,0.2509,0.0002,0.0131,0.0002,0.0000


(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
Query time: 0.01591658592224121
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -0.10234367908719832
Internal consistency (silhouette, correlation) for unfiltered: 0.9797015821157272
Internal consistency (Calinski&Harabasz)  for unfiltered: 103.87302198018776
Internal consistency (Davies&Bouldin) for unfiltered: 0.1824520608063022


index = ['emotion']
{'emotion', 'intensifier'} {'intensifier'} {'emotion'}
[]


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
cheerful,0.0000,0.0255,0.0259,0.0037,0.2371,0.2065,0.2653,0.2361
happy,0.0000,0.0310,0.0392,0.0017,0.2405,0.2383,0.2258,0.2235
hopeful,0.0000,0.0465,0.0517,0.0179,0.2106,0.2251,0.1976,0.2506
depressed,0.2157,0.2080,0.2236,0.2435,0.0315,0.0623,0.0151,0.0002
down,0.2167,0.2762,0.2401,0.2614,0.0001,0.0054,0.0000,0.0000
hopeless,0.3394,0.1886,0.2047,0.2583,0.0023,0.0049,0.0018,0.0000


In [ ]:
print('test')

In [10]:
Q()._scale

'intensifier'